# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
import re 
import nltk
import warnings
import random 
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize, WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
#from sklearn.multiclass import OneVsRestClassifier
#from sklearn.externals import joblib
from sklearn.svm import LinearSVC
nltk.download(['punkt', 'wordnet'])
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///project5.db')
df = pd.read_sql_table('MSG', engine)
X = df['message']
Y = df.iloc[:, 4:]
cate = Y.columns

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    #lets get tokens
    token = WhitespaceTokenizer().tokenize(text)
    #token = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    # lets clean tokens and return them
    cleaned_tk = []
    for i in token:
        i = lemmatizer.lemmatize(i).lower().strip('!@#$%^&*()"/+=[\\]_-{|}~\'`,;.:<>?')
        i  = re.sub(r'\[[^,.;:]*\]' , '' , i )
        if i != '':
            cleaned_tk.append(i)
            
    return cleaned_tk

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf',TfidfTransformer() ),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
random.seed(20181022)

In [6]:
X_train, X_test, y_train, y_test  = train_test_split(X, Y
                                                     #, test_size = 0.25, random_state=50
                                                    )

In [7]:
#error resolved using reference 
#https://stackoverflow.com/questions/50311793/valueerror-found-input-variables-with-inconsistent-numbers-of-samples-7111-1

pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [8]:
y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
print(classification_report(y_test.iloc[:,1:].values, np.array([x[1:] for x in y_pred]), target_names= cate))

                        precision    recall  f1-score   support

               related       0.81      0.40      0.54      1059
               request       0.00      0.00      0.00        33
                 offer       0.75      0.52      0.62      2722
           aid_related       0.62      0.09      0.15       559
          medical_help       0.73      0.09      0.16       327
      medical_products       0.75      0.05      0.09       197
     search_and_rescue       0.20      0.01      0.02       105
              security       0.62      0.06      0.11       216
              military       0.00      0.00      0.00         0
           child_alone       0.79      0.27      0.40       368
                 water       0.83      0.29      0.44       695
                  food       0.82      0.23      0.35       584
               shelter       0.67      0.02      0.04        96
              clothing       0.75      0.02      0.03       168
                 money       1.00      

In [10]:
#cv = GridSearchCV(pipeline, parameters)

In [11]:
#cv.fit(X_train, y_train)

### 6. Improve your model
Use grid search to find better parameters. 

In [12]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf__estimator__random_state', 'clf__estimator__verbose', 'clf__estimator__

In [13]:
parameters = {'clf__estimator__max_depth': [10, 50, None],
              'clf__estimator__min_samples_leaf':[2, 5, 10]}

In [14]:
cv =  GridSearchCV(pipeline, param_grid= parameters #verbose=2, scoring= 'f1_weighted'
                  )

In [15]:
cv.get_params().keys()

dict_keys(['cv', 'error_score', 'estimator__memory', 'estimator__steps', 'estimator__vect', 'estimator__tfidf', 'estimator__clf', 'estimator__vect__analyzer', 'estimator__vect__binary', 'estimator__vect__decode_error', 'estimator__vect__dtype', 'estimator__vect__encoding', 'estimator__vect__input', 'estimator__vect__lowercase', 'estimator__vect__max_df', 'estimator__vect__max_features', 'estimator__vect__min_df', 'estimator__vect__ngram_range', 'estimator__vect__preprocessor', 'estimator__vect__stop_words', 'estimator__vect__strip_accents', 'estimator__vect__token_pattern', 'estimator__vect__tokenizer', 'estimator__vect__vocabulary', 'estimator__tfidf__norm', 'estimator__tfidf__smooth_idf', 'estimator__tfidf__sublinear_tf', 'estimator__tfidf__use_idf', 'estimator__clf__estimator__bootstrap', 'estimator__clf__estimator__class_weight', 'estimator__clf__estimator__criterion', 'estimator__clf__estimator__max_depth', 'estimator__clf__estimator__max_features', 'estimator__clf__estimator__max

In [16]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f0523b6d9d8>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [17]:
cv.fit(X_train.as_matrix(), y_train.as_matrix())

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__max_depth': [10, 50, None], 'clf__estimator__min_samples_leaf': [2, 5, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [18]:
y_pred = cv.predict(X_test)

In [19]:
print(classification_report(y_test.iloc[:,1:].values, np.array([x[1:] for x in y_pred]), target_names= cate))

                        precision    recall  f1-score   support

               related       0.87      0.30      0.44      1059
               request       0.00      0.00      0.00        33
                 offer       0.74      0.63      0.68      2722
           aid_related       0.70      0.06      0.11       559
          medical_help       0.62      0.02      0.05       327
      medical_products       0.50      0.03      0.05       197
     search_and_rescue       1.00      0.01      0.02       105
              security       0.70      0.03      0.06       216
              military       0.00      0.00      0.00         0
           child_alone       0.86      0.07      0.13       368
                 water       0.80      0.20      0.32       695
                  food       0.77      0.16      0.27       584
               shelter       1.00      0.03      0.06        96
              clothing       0.00      0.00      0.00       168
                 money       0.00      

In [20]:
cv.best_estimator_

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [21]:
# testing done in the previous step ;)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [22]:
abc = MultiOutputClassifier(AdaBoostClassifier())

pipeline_2 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', abc)
    ])

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [24]:
pipeline_2.fit(X_train.as_matrix(), y_train.as_matrix())

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))])

In [25]:
y_pred = pipeline_2.predict(X_test)

In [26]:
print(classification_report(y_test.iloc[:,1:].values, np.array([x[1:] for x in y_pred]), target_names= cate))

                        precision    recall  f1-score   support

               related       0.80      0.53      0.64      1128
               request       0.00      0.00      0.00        29
                 offer       0.78      0.58      0.66      2780
           aid_related       0.63      0.25      0.36       533
          medical_help       0.60      0.32      0.41       317
      medical_products       0.52      0.16      0.25       180
     search_and_rescue       0.17      0.03      0.06       122
              security       0.66      0.37      0.47       213
              military       0.00      0.00      0.00         0
           child_alone       0.75      0.63      0.68       445
                 water       0.82      0.67      0.74       768
                  food       0.78      0.52      0.63       566
               shelter       0.70      0.43      0.53       102
              clothing       0.44      0.24      0.31       131
                 money       0.72      

### 9. Export your model as a pickle file

In [27]:
import pickle

pickle.dump(cv, open('model.pkl','wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.